# Importação de bibliotecas

In [2]:
# importando bibliotecas necessárias
from dotenv import load_dotenv
import os
from groq import Groq
import re

# carregando arquivo de variáveis de ambiente com chave da api
load_dotenv()
print("importação funcionou")

importação funcionou


# Classes necessárias para o agente

In [3]:
# instanciando a ferramenta de serviço groq para utilização do modelo LLM llama-3.3
# testando o modelo

cliente = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

chat_completion = cliente.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Explique em 10 palavras o que é IMC",
        }
    ],
    model="llama-3.3-70b-versatile",
)

print(chat_completion.choices[0].message.content)


# modelo de agente desenvolvido por https://www.youtube.com/watch?v=hKVhRA9kfeM
# aqui é o modelo de agente responsável por executar a cadeia de pensamentos e interações entre o usuário e também entre o agente
class Agente:
    def __init__(self, cliente: Groq, prompt_inicial: str = "") -> None:
        self.cliente = cliente
        self.prompt_inicial = prompt_inicial
        self.mensagens: list = []
        if self.prompt_inicial:
            self.mensagens.append({"role": "system", "content": prompt_inicial})

    def __call__(self, mensagem=""):
        if mensagem:
            self.mensagens.append({"role": "user", "content": mensagem})
        resultado = self.execute()
        self.mensagens.append({"role": "assistant", "content": resultado})
        return resultado

    def execute(self):
        completion = cliente.chat.completions.create(
            model="llama3-70b-8192", messages=self.mensagens
        )
        return completion.choices[0].message.content


# aqui é o prompt inicial, ou prompt do sistema
prompt_inicial = """
Você executará uma sequência de estados começando por <Pensamento>, <Ação>, <Pausa> e <Observação>.
No final da sequência a sua saída será uma <Resposta> e uma <Dica> de saúde.
Use o estado <Pensamento> para descrever seu raciocínio sobre uma pergunta.
Use <Ação> para executar um das tarefas disponíveis para você e então retorne <Pausa>.
A <Observação> será o resultado de uma execução das tarefas.

As ações disponivéis são:

calcular_imc:
exemplo, calcular_imc: (peso, altura)
Execute o cálculo e retorne o número do IMC - a linguagem utilizada será python com pontos flutuantes


classificacao_imc:
exemplo, classificacao_imc: 18,5
retorne a classificacao do IMC

Sessão de exemplo:
Pergunta: Qual o meu IMC?
Pensamento: qual seu peso e altura?
Ação: calcular_imc: 85, 1.70
Pausa

Uma nova interação será realizada assim:

Observação: 29.41

Pensamento: Preciso classificar o  IMC
Ação: classificacao_imc: 29.41
Pausa

Uma nova interação será realizada assim:

Observação: SOBREPESO

Se você tem a resposta, retorne ela como <Resposta>.

<Resposta>: O IMC do seu peso e altura está classificado como Sobrepeso, faça dieta e exercícios!

Agora é sua vez:
""".strip()

# ferramenta utilizada para calcular o IMC com base nos dados fornecidos pelo usuário
def calcular_imc(args) -> float:
    '''
    responsavel por calcular o IMC do seu peso e altura através da fórmula peso / altura * altura
    :param args:
    :return:
    '''
    args = args.split(",")
    peso = float(args[0])
    altura = float(args[1])
    retorno = peso / (altura * altura)
    return retorno

# ferramenta utilizada para classificar o IMC
def classificacao_imc(imc: str) -> str:
    '''
    responsável por classificar o IMC do seu peso e altura com base na tabela de IMC
    :param imc:
    :return:
    '''
    imc = float(imc)
    if imc < 18.5:
        return "MAGREZA"
    if imc >= 18.5 and imc <= 24.9:
        return "NORMAL"
    if imc >= 25 and imc <= 29.9:
        return "SOBREPESO"
    if imc >= 30 and imc <= 39.9:
        return "OBESIDADE"
    if imc >= 40:
        return "OBESIDADE GRAVE"

# teste unitário das funções, teste em cadeia, primeiro calculando o IMC e posteriormente classificando
print(classificacao_imc(calcular_imc("92, 1.99")))


Índice de massa corporal para medir gordura do corpo.
NORMAL


# testando o agente criado

In [4]:
# testando o agente e o prompt do sistema criado
# instanciação de teste do agente

agente_imc_teste = Agente(cliente=cliente, prompt_inicial=prompt_inicial)

# testando o estado inicial do agente
resposta_do_agente = agente_imc_teste.execute()
print(resposta_do_agente)


resposta_do_agente = agente_imc_teste("Me chamo paulo, meu peso é 85 kilos e minha altura é 1,70 metros")
print(resposta_do_agente)

Pensamento: Qual é o seu peso e altura?
Olá Paulo!

<Pensamento>: Para calcular o IMC, preciso do peso e altura. Você me forneceu essas informações, então posso calcular o IMC.

<Ação>: calcular_imc: 85, 1.70
<Pausa>


# Criando a interação do usuário e a iteração do agente

In [7]:
# executando o agente

# definindo as ferramentas
ferramentas = ["calcular_imc", "classificacao_imc"]


# definindo o laço de repetição do agente
def iteracao(repeticao = 10, prompt: str = "", verbose: bool = False):
    '''
    função responsavel por realizar a iteração com o agente desenvolvido
    a função recebe um prompt incial e recebe um parametro de verbose, utilizado para imprimir os logs da solução
    A função é executada até que o processo finalize através do ponto de parada, que é a geração da resposta esperada ou a quantidade máxima de iterações.
    O prompt inicial é armazenado em uma lista, que simula uma memória do agente. Os próximos prompts e resultados também são armazenados.
    Através do promtp, o agente também consegue executar ferramentas personalizadas e incluir essas repostas em prompts futuros
    :param repeticao:
    :param prompt:
    :param verbose:
    :return:
    '''
    agente = Agente(cliente=cliente, prompt_inicial=prompt_inicial)
    proximo_prompt = prompt

    indice = 0

    # condição de parada por exaustão
    while indice < repeticao:
        indice += 1
        resposta = agente(proximo_prompt)
        if verbose: print(resposta)

        if "<Pausa>" in resposta and "<Ação>" in resposta:
            action = re.findall(r"<Ação>: ([a-z_]+): (.+)", resposta, re.IGNORECASE)
            if verbose: print(action)
            ferramenta = action[0][0]
            if verbose: print(ferramenta)
            parametros = action[0][1]
            if verbose: print(parametros)


            if ferramenta in ferramentas:
                resultado_ferramenta = eval(f"{ferramenta}('{parametros}')")
                proximo_prompt = f"<Observação>: {resultado_ferramenta}"
            else:
                proximo_prompt = "<Observação>: Ferramenta não existente"

            if verbose: print(proximo_prompt)
            continue

        # condição de parada por chegar na resposta esperada
        if "<Resposta>" in resposta:
            print(resposta)
            break

# executando o agente
iteracao(prompt="Me chamo Paulo, meu peso é 81 kilos e minha altura é 1,99 metros. Qual meu IMC?", verbose=False)

<Resposta>: O IMC do seu peso e altura está classificado como NORMAL, parabéns! Você está dentro do peso saudável.

<Dica>: Continue a se alimentar bem e a se exercitar regularmente para manter sua saúde em dia!
